In [1]:
from glob import glob
from tqdm import tqdm
import json
import random

In [2]:
rejected = [
    'help.openai.com',
    'openassistant'
]

break_at = [
    'am an ai',
    'unable to view',
    'translate',
    'keyword',
    'assistant',
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'am sorry',
    'sorry',
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'saya tidak mempunyai',
    'saya tidak pernah',
    'saya tidak dapat memahami jawapan',
    '=====',
    '-----',
    'tidak faham bahasa melayu',
    'tidak faham bahasa inggeris',
    'sehari-hari',
]

def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return True, words[i]
    return False, None

def accept(d, min_len = 10):
    
    d = d.strip()
    
    if len(d.split()) < min_len:
        return False
        
    if 'terjemahkan teks' in d.lower():
        return False

    if 'no need to translate' in d.lower():
        return False

    if 'can be translated' in d.lower():
        return False

    if 'cannot translate' in d.lower():
        return False

    if 'should be translated to' in d.lower():
        return False

    if 'cannot be translated' in d.lower():
        return False

    if 'standard malay' in d.lower():
        return False

    if 'would not be translated' in d.lower():
        return False

    if 'as an AI language model' in d:
        return False

    if 'should be translated as' in d.lower():
        return False

    if 'Bahasa Malaysia Standard' in d:
        return False
    
    if 'Saya adalah model AI' in d:
        return False
    
    if 'saya model AI' in d:
        return False
    
    if 'Saya model AI' in d:
        return False
    
    if 'sebagai model AI' in d:
        return False
    
    if 'Sebagai model AI' in d:
        return False
    
    if 'model bahasa AI' in d:
        return False
    
    if 'model AI yang dibangunkan' in d:
        return False
    
    if d == '<s>':
        return False
    
    if 'tidak dapat memberikan maklumat' in d:
        return False
    
    if 'Sebagai model bahasa' in d:
        return False
    
    d_lower = d.lower()
    splitted = d_lower.split()
    if (len(set(splitted)) / len(splitted)) < 0.2:
        return False
    
    if found_word(d_lower, rejected)[0]:
        return False
    
    if found_word(d_lower, break_at)[0]:
        return False
    
    filtered_string = ''.join(char for char in d if char.isascii())
    if len(filtered_string) < 10:
        return False
    
    return True

In [3]:
similar = [
    {"text": "describe every detail in the picture"},
    {"text": "what do you see in the image"},
    {"text": "provide a detailed description of the photo"},
    {"text": "tell me everything about what's happening in the image"},
    {"text": "describe the scene depicted in the picture"},
    {"text": "what's going on in the image"},
    {"text": "give me a detailed account of what's in the photo"},
    {"text": "describe the visual elements in the image"},
    {"text": "what do you notice about the picture"},
    {"text": "describe the content of the image"},
    {"text": "paint a picture with words about the image"},
    {"text": "describe the image from top to bottom"},
    {"text": "what's the main subject of the image"},
    {"text": "what's the context of the image"},
    {"text": "recreate the image with words"},
    {"text": "describe the image in exquisite detail"},
    {"text": "what's the narrative behind the picture"},
    {"text": "give a thorough examination of the image"},
    {"text": "describe the visual storytelling in the photo"},
    {"text": "what's the central theme of the image"},
    {"text": "what's the most intriguing aspect of the photo"},
    {"text": "walk me through the image element by element"},
]
similar = [s['text'] for s in similar] + ['describe everything about this image']

In [4]:
files = glob('output-vehicle/*.json')
len(files)

4068

In [5]:
with open('filtered-vehicle.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)

        for d in data:
            questions, answers = [], []
            for no, q in enumerate(d['questions']):
                if no > 0 and not accept(q):
                    continue

                if not accept(d['answers'][no]):
                    continue

                if no == 0:
                    q = random.choice(similar)

                questions.append(q)
                answers.append(d['answers'][no])

            if not len(questions):
                continue

            d['questions'] = questions
            d['answers'] = answers

            fopen_l.write(f'{json.dumps(d)}\n')
            fopen_l.flush()

100%|██████████| 4068/4068 [00:33<00:00, 120.38it/s]


In [6]:
!wc -l filtered-vehicle.jsonl

96949 filtered-vehicle.jsonl


In [7]:
!head -n 1 filtered-vehicle.jsonl

{"filename": "image-vehicle/train-00027-of-00165-c7635e0fc5e84b30.parquet-4538.jpg", "filename_df": "train-00027-of-00165-c7635e0fc5e84b30.parquet", "keyword": "2022 Mercedes-Benz GLC Coupe 300 AMG Line(CKD)", "alt_text": "Image 2 details about Will there be a ...", "questions": ["What specific design features make the Mercedes-Benz GLC Coupe 300 AMG Line (CKD) stand out in the image?", "What type of tire and rim design is featured on the white Mercedes-Benz SUV?"], "answers": ["The 2022 Mercedes-Benz GLC Coupe 300 AMG Line (CKD) is shown in an image that highlights its sleek and luxurious design. The vehicle exhibits a sleek design with sharp lines and contours, showcasing the AMG Line (CKD) designation, which typically represents high performance and exclusive features. The car's coupe style is evident from its two-door configuration and sporty roofline. The wheels of the car are multi-spoke alloy wheels, adding to its stylish appearance. The vehicle is painted in a metallic hue, and

In [8]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj='filtered-vehicle.jsonl',
    path_in_repo='filtered-vehicle.jsonl',
    repo_id='mesolitica/google-image-malaysian-vehicle-semisupervised',
    repo_type='dataset',
)

filtered-vehicle.jsonl:   0%|          | 0.00/237M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/google-image-malaysian-vehicle-synthetic-qa/commit/cb4688250e88e0d53938e4c448bd4d528b9b19d3', commit_message='Upload filtered-vehicle.jsonl with huggingface_hub', commit_description='', oid='cb4688250e88e0d53938e4c448bd4d528b9b19d3', pr_url=None, pr_revision=None, pr_num=None)